In [ ]:
import pandas as pd
from services.jira_service import JiraService  # seu service acima
from datetime import datetime
from services.jira_parser import parse_issues_to_dataframe
import json


jira = JiraService()
board_id = 71  # atualize se necessário

backlog_data = jira.get_raw_backlog_issues(board_id).get("issues", [])
backlog_df = parse_issues_to_dataframe(backlog_data)




In [ ]:
print(len(backlog_data))

In [ ]:
issues_data = []
for issue in backlog_data:
    #if issue["key"] in ["SQD-690", "SQD-1012"]:

    if issue["fields"]["issuetype"]["name"] != "Subtarefa":
        print(issue["key"], issue["fields"]["issuetype"]["name"])


#     fields.get("issuetype") or {}).get("name", ''),
# #     if issue["key"] in ["SQD-690", "SQD-1012"]:
# #         print(issue["key"])
# #         issues_data.append(issue)

# with open('sprint_data_teste.json', 'w', encoding='utf-8') as f:
#     json.dump(issues_data, f, ensure_ascii=False, indent=4)

  
